In [ ]:
# Copy and paste all required information here for the listed variables, make sure to paste inside the quotation marks

# Your API_KEY from My Settings tab in Planet Labs account
API_KEY = '' 

# Your GCS project ID in which your bucket is located
GCS_PROJECT_ID = '' 

# Your GCS bucket name that resides in the GCS project
GCS_BUCKET_NAME = '' 

# The file name where you saved your GCS credentials, be sure to include .json
GCS_CREDS_JSON = '' 

# The file name where you saved your AOI information, be sure to include .geojson
GEOJSON_FILE = '' 

# The start and end date for monitoring format YYYY-MM-DD
START_DATE = '' 
END_DATE = ''

# Give the subscription a name
SUB_NAME = ''

In [ ]:
# import required libraries

import getpass
import planet
from planet import Auth

import asyncio
import os
from planet import Session


import json
import base64
import requests
from requests.auth import HTTPBasicAuth

import pathlib


In [ ]:
# base url for subscriptions
BASE_URL = "https://api.planet.com/subscriptions/v1"

In [ ]:
# Confirm that your API key is valid

# Set an authentication value
auth = HTTPBasicAuth(API_KEY, '')

# Do a GET request and the response
response = requests.get(BASE_URL, auth=auth)
# response 200 means its working
print(response)

# if you get a response other than 200 check your API_KEY

In [ ]:
# Load GCS service agent's JSON credentials as Base64
with open(GCS_CREDS_JSON) as jsonfile:
  data = json.load(jsonfile)
  datastr = json.dumps(data)
  msg_bytes = datastr.encode('ascii')
  base64_bytes = base64.b64encode(msg_bytes)
  decoded = base64_bytes.decode('ascii')
  GCS_CREDS_B64 = decoded

In [ ]:
# load aoi through files
with open(GEOJSON_FILE) as f:
  geojson_data = json.loads(f.read())

In [ ]:
# putting start and end date in correct format
START_DATE_FORMAT = START_DATE + 'T00:00:00Z'
END_DATE_FORMAT = END_DATE + 'T00:00:00Z'

In [ ]:
# create filters

# images that overlap with AOI
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geojson_data["features"][0]["geometry"]
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "published",
  "config": {
    "gte": START_DATE_FORMAT,
    "lte": END_DATE_FORMAT
  }
}

# only get images which have <=50% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5 
  }
}

# combine filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter,  date_range_filter, cloud_cover_filter]
}

In [ ]:
sub_request = {
    # Name Block
    "name": SUB_NAME,
    # Source Block
    "source": {
        "type": "catalog",
        "parameters": {
            "item_types": ["PSScene"],
            "asset_types": ["ortho_analytic_4b_sr", "ortho_udm2", "ortho_analytic_4b_xml"],
            #There's no limit to the types of searches you can create with the filter functionality.
            "filter": combined_filter
        }
    },

    "tools": [
        {
            'type': 'harmonize',
            'parameters': {
                'target_sensor': 'Sentinel-2'
            }
        },
        {
          "type": "clip",
          "parameters": {
              "aoi": geojson_data["features"][0]["geometry"]
            }
        }
    ],

    #Delivery
    "delivery": {
        "type": "google_cloud_storage",
        "parameters": {
            "bucket": GCS_BUCKET_NAME,
            "credentials": GCS_CREDS_B64
        }
    }
}

In [ ]:
# send initial request, prints out information about the subscription

sub_response = requests.post(BASE_URL, json=sub_request, auth=(API_KEY, ''))
print('Order Status Code: ', sub_response.status_code)
order_id = sub_response.json()['id']
print('Order ID: ', sub_response.json()['id'])
print('Full Order Info: ', sub_response.json())

In [ ]:
# # url for current subscription, uncomment if you want to edit or cancel your subscription
# sub_url = BASE_URL + '/' + order_id
# print(sub_url)

In [ ]:
# # edit subscription
# # edit any variables above then run this code to edit the subscription
# # if you are chaning AOI just create a new subscription instead
# sub_revised_response = requests.put(sub_url, json=sub_request, auth=(API_KEY, ''))
# print('Order Status Code: ', sub_revised_response.status_code)
# print('Full Order Info: ', sub_revised_response.json())

In [ ]:
# # cancel subscription
# # run this to stop monitoring your AOI
# cancel_url = BASE_URL + '/' + order_id + '/cancel'
# sub_cancel_response = requests.post(cancel_url, auth=(API_KEY, ''))
# print('Order status code: ', sub_cancel_response.status_code)


In [ ]:
# # View running subscriptions on your Planet Labs Account
# test_url = BASE_URL + '?status=running&status=completed'
# sub_test_response = requests.get(test_url, auth=(API_KEY, ''))
# print('Order status code: ', sub_test_response.status_code)
# print('All Info: ', sub_test_response.json())